In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import warnings
warnings.filterwarnings('ignore')

Import Data

In [3]:
df = pd.read_csv('/content/drive/My Drive/yelp_review_arizona.csv')
df.head()

,review_id,user_id,business_id,text,stars,date
0,V93SYj2OLh5m9Cquzf-7kg,ZwVz20be-hOZnyAbevyMyQ,2c9Vptks_vowLgVUMnCgjw,Came here while in town for a country concert....,4.0,2013-09-04 01:29:46
1,vNTFadc6T9HeH3Qa78dc_Q,91TB-gzcNyxFh46TL0pmnQ,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue this side of the Mississippi!!!!...,5.0,2015-12-05 02:50:10
2,SXRFBCt5eXCBF7TlI7UG6Q,Y_QBiZpATJoz8hKUfYF66A,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing. Think Chipotle for enchila...,5.0,2014-04-01 01:56:00
3,CqMNjtG0hNZGhDw4RDE-zw,_Jg-IA0M-GSjBlGu-wmejg,r8764MtYyt8JhxMvrfM_xQ,I was really disappointed with my most recent ...,2.0,2014-10-11 03:53:53
4,5hZLouGEW4wm6BTJ5aNUNw,1CqkFliipv_X15WYn5aPfg,QS3QxI7u5PRdtbGgI0-UsA,I grade sushi restaurants on 3 factors:\n- Qua...,4.0,2015-03-04 19:36:21


In [4]:
df.shape

(26606, 6)

Select only stars and text

In [5]:
df_business = pd.read_csv('/content/drive/My Drive/yelp_business.csv')
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",NaN
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,NaN,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [6]:
#Check Null values in Dataframe
df.isnull().sum()

review_id      0
user_id        0
business_id    0
text           0
stars          0
date           0
dtype: int64

In [0]:
#Select only stars and text
yelp_data = df[['business_id', 'user_id', 'stars', 'text']]

In [8]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])

Clean Text

In [0]:
yelp_data['text'] = yelp_data['text'].apply(text_process)


In [0]:
#Split train test for testing the model later
vld_size=0.15
X_train, X_valid, y_train, y_valid = train_test_split(yelp_data['text'], df['business_id'], test_size = vld_size)

Create two tables of user,text and business,text

In [0]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [13]:
userid_df.head()

,user_id,text
0,ZwVz20be-hOZnyAbevyMyQ,Came town country concert better way start day...
1,91TB-gzcNyxFh46TL0pmnQ,Best barbecue side Mississippi Come car washed...
2,Y_QBiZpATJoz8hKUfYF66A,Absolutely amazing Think Chipotle enchiladas N...
3,_Jg-IA0M-GSjBlGu-wmejg,really disappointed recent visit Vintage 95 ma...
4,1CqkFliipv_X15WYn5aPfg,grade sushi restaurants 3 factors Quality Exce...


In [14]:
userid_df[userid_df['user_id']=='ZwVz20be-hOZnyAbevyMyQ']['text']

0        Came town country concert better way start day...
1098     Amazing Im golfer ended loving went family lov...
3909     saw many great reviews place decided try high ...
11491    Amazing Mexican Asian infusion good service sa...
19647    beautiful hotel excellent service Nick front a...
Name: text, dtype: object

In [15]:
business_df.head()

,business_id,text
0,2c9Vptks_vowLgVUMnCgjw,Came town country concert better way start day...
1,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue side Mississippi Come car washed...
2,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing Think Chipotle enchiladas N...
3,r8764MtYyt8JhxMvrfM_xQ,really disappointed recent visit Vintage 95 ma...
4,QS3QxI7u5PRdtbGgI0-UsA,grade sushi restaurants 3 factors Quality Exce...


Join the text with for each user_id and business_id

In [0]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

In [17]:
userid_df.head()

,text
user_id,
--2HUmLkcNHZp0xw6AMBPg,place JAM Surfer vibe great eats love machaca ...
--4rAAfZnEIAKJE80aIiYg,pulled pork spicy bbq sauce impressed Probably...
--Nnm_506G_p8MxAOQna5w,Cant say burger anything special Taste ok Shak...
--ty7Z9fEt08E3dS3_qoSA,know think important trust Yelp kind reviews t...
-0IiMAZI2SsQ7VmyzJjokQ,Ever fan roadside attractions Americana Id rea...


In [18]:
userid_df.loc['ZwVz20be-hOZnyAbevyMyQ']['text']

'Came town country concert better way start day Everything great service amazing time walked door sat table surprise beer whiskey tap table pay ounce screen show drank food great whiskey burger favorite country fans must Scottsdale Amazing Im golfer ended loving went family loves golf planned staying hour since golf 6 hours later left far drive place right food amazing expect good brisket tacos chorizo sliders favorite Good quality food many unique items highly recommend place people ages teens loved bays nice roomy 7 us bay 10 people must try back soon saw many great reviews place decided try high hopes loved atmosphere kids loved Texas trivia game thats good thing could say steak came cooked way asked took back fix steak flavor covered butter thought steak even kids eat theres waitress asked daughter something wrong since ate bites explained like said ok walked away shrimp good pricey bugers ok end pushing us take home since ate almost none politely declined still came boxes trying g


User Tfidf Vectorizer with 5000 Features (represent 88% of all words)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])
userid_vectors.shape

(10937, 5000)

In [21]:
userid_vectors

<10937x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 1083887 stored elements in Compressed Sparse Row format>

In [22]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])
businessid_vectors.shape

(1411, 5000)

Matrix Factorization

In [23]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])
userid_rating_matrix.shape

(10937, 1411)

In [24]:
userid_rating_matrix.head()

business_id,-050d_XIor1NpCuWkbIVaQ,-1UMR00eXtwaeh59pEiDjA,-4TMQnQJW1yd6NqGRDvAeA,-6h3K1hj0d4DRcZNUtHDuw,-8QlV3b_9H4BAh6LgMIr1g,-9eNGMp8XiygI8t8QFuFWw,-9nai28tnoylwViuJVrYEQ,-Bdw-5H5C4AYSMGnAvmnzw,-BxWyEIQ6wypT-37MzZizQ,-CfFjcCcGGDM9MVH_d42RQ,-F5mm0-YeCI7viSiOwVAAw,-FLnsWAa4AGEW4NgE8Fqew,-Ht7HiGBox8lS1Y8IPjO8g,-ITj6Pu8Gdw8MmLf0XBEKQ,-JwSuDqyFILFJmVpjrVbwA,-KIdCJnkt5N8rnnmWR5MQg,-Ma5JqzuI_3xIAtpNJfQ2g,-MuatiMmslPOvk9kOMyjkA,-PUPEpMsjBjk_rX5lFaWng,-RYIKWBSTaJ21B2UMyzI-Q,-ZBfr1BHvArFp1d6XH8jOQ,-bMZCfTK7fxFaURynKpBMA,-bd4BQcl1ekgo7avaFngIw,-cBQKodqi77Q0vk-9iDvQA,-hq09PjRpGcx4XXhrqh5xw,-kIscN8I29eXMPkvyyxmRQ,-o082vExIs0VVNSuZmiTQA,-oSII3bw90cvyLmgsHgmpg,-rCIoxp7pqN3xY7E3-v56Q,-raUM9T-1dKPiB5_vRmFYA,-rhH9sL3XGFpoJXcxUpEWA,-s5zZEv5u9XWhx1i5E0_pA,-s_U95xEEmJu5GK-srqpUA,-vCLrTTgw6pBufdarW8ynA,-yQHIYKXH3HAdhh1W52OMQ,01xTdrNUuTOAyH7NaRWcUA,024QUHVU7nlyW5y2BeeGLg,02EYqwh47uVxOaOKrnx8SQ,042IHd5KjHiMuBtGtugO_g,05BpxQYVbywxliQ9yA0CKw,...,y2e1DgIUBlERep7hrKz2-w,yA6dKNm_zl1ucZCnwW8ZCg,yAXWKWRTs4wNy-zrJgoeXQ,yCK1Ok69D7TSWJESZQZHiQ,yF9G4dJAm94pfnEWzPtPYQ,yFUvKpW66cNsYXD6inDsog,yLMSxHjK56Az-KtMQyG7LQ,yNPh5SO-7wr8HPpVCDPbXQ,yRvmDKX-Ocgd8zq_iMgujA,yUkOP3Tmdp1pWA7p3zHboQ,yX0pOWG_Be9TjGnm0aihVA,yXj1P-pc6CGDFSta-45L5Q,yZAGhlsrmupLbrgdKm6reg,ydqbNz2sHENV9XOElwyAYA,ydt5yifyYhF5s430llv_Hw,yet0Pww_ouW49T7V41ecSw,ylxqmxh2gO1yCpQkIk6o3A,ymtJtXwsJUClLNDepV1piQ,yqYtY3-Po4OVPafA9Z-Xyw,ytAsW7DYezxMMzqoO_-nIQ,yvwK4MO2qnSZE5ggYGyk1A,z4K95Y4P5nMrtJiDtXWDYA,z5KWNSDvgv-4I62P8l8Ynw,z6-reuC5BYf_Rth9gMBfgQ,zDyv4CoaYvAgQhgr1IK1sg,zJGtD3y-pAIGNId4codEEg,zNGYsFeG_7XEBodbtfux5Q,zOL260xaXDYZZPBItzALxQ,zS3-CKZVVvUYvasJnAg-aA,zUgDrRtGvK5ZTFlHCsTHwA,z_lDO8d8nkSmcvTjB4N69A,za9qr9ZZWLTfEgTfogRbUw,zbrFk-4ejesAJD8EwcdHxg,zfiSQ1dl3vTJ-og96eqXGA,zidkKI_N1OPxsiddTOQH_Q,zqNgwQjj0_XAll-neGikIw,zr93wrNyXzc-HW4IcK4iRQ,zrDi4gEaUi64lAMfJU51dw,zrTGcb83AsfyVTMrsCa65A,zuVvDYJkKAbXQTTBauAqJQ
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--2HUmLkcNHZp0xw6AMBPg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--4rAAfZnEIAKJE80aIiYg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Nnm_506G_p8MxAOQna5w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ty7Z9fEt08E3dS3_qoSA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-0IiMAZI2SsQ7VmyzJjokQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())

In [1]:

Q.head()

NameError: ignored

Gradient Decent Optimization

In [0]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q

In [28]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

CPU times: user 3h 3min 49s, sys: 2.54 s, total: 3h 3min 52s
Wall time: 3h 4min 20s


In [29]:
Q.head()

,0,1,10,100,1000,101,1015,1030,10pm,11,110,1100,11am,12,13,14,15,150,1520,16,16th,17,18,19,1st,2,20,200,2030,21,22,23,24,25,250,26,27,2nd,2pm,3,...,yall,yard,yasu,yay,yea,yeah,year,years,yell,yelling,yellow,yellowtail,yelp,yelper,yelpers,yep,yes,yesterday,yet,yikes,yo,yogurt,york,yorker,young,younger,yuck,yum,yummy,yup,z,zero,zest,zinburger,zinc,zipps,ziti,zookz,ztejas,zucchini
business_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-050d_XIor1NpCuWkbIVaQ,0.001158,0.046774,0.043362,0.041756,0.006478,0.002138,0.010029,0.023638,0.000542,0.001409,0.000750,0.033710,0.000435,0.020155,0.001093,0.001206,0.033249,0.001074,0.027486,0.003051,0.017109,0.000492,0.001061,0.000407,0.046995,0.078744,0.034059,0.000749,0.000209,0.005676,0.000637,0.000800,0.000995,0.013523,0.058090,0.000293,0.000438,0.001036,0.000166,0.075108,...,0.024777,0.000699,0.000140,0.006370,0.010359,0.042611,0.025279,0.035230,0.000220,0.000566,0.004363,0.002199,0.035735,0.000564,0.011619,0.001304,0.087830,0.002010,0.056844,0.005791,0.000323,0.001960,0.012342,0.000417,0.007704,0.000813,0.000850,0.044001,0.098376,0.000384,0.000139,0.000476,0.000238,0.013895,0.000119,0.000197,0.001806,0.000067,0.000085,0.004687
-1UMR00eXtwaeh59pEiDjA,0.000126,0.001169,0.001863,0.000566,0.000278,0.000095,0.000099,0.000047,0.000095,0.000339,0.000114,0.000283,0.000022,0.000701,0.000158,0.000362,0.002237,0.000238,0.000340,0.000198,0.000019,0.000204,0.000177,0.000334,0.000170,0.047089,0.002468,0.000016,0.000009,0.000121,0.000020,0.000258,0.000179,0.169652,0.000141,0.000059,0.000014,0.000248,0.000008,0.045083,...,0.000106,0.000176,0.000004,0.000100,0.000016,0.000490,0.000769,0.004019,0.000022,0.000017,0.000529,0.000043,0.001167,0.000020,0.000241,0.000010,0.003548,0.000284,0.004084,0.000029,0.000004,0.000161,0.000353,0.000022,0.000705,0.002011,0.000081,0.000669,0.000921,0.000024,0.000003,0.000444,0.000181,0.000015,0.000010,0.000197,0.000387,0.000018,0.000008,0.000711
-4TMQnQJW1yd6NqGRDvAeA,0.000497,0.004620,0.003707,0.001494,0.000228,0.000101,0.000558,0.000692,0.000350,0.001170,0.000325,0.000743,0.000147,0.004869,0.004051,0.000878,0.005298,0.000445,0.000107,0.000519,0.000582,0.000377,0.000426,0.000056,0.000760,0.009670,0.005698,0.000850,0.000342,0.000626,0.000154,0.000461,0.000697,0.001180,0.000411,0.000085,0.000135,0.000734,0.000374,0.014002,...,0.000129,0.001398,0.000058,0.000200,0.000282,0.001330,0.055112,0.007509,0.004763,0.000413,0.001454,0.000751,0.004840,0.000235,0.000804,0.012710,0.050755,0.001186,0.010724,0.000217,0.000570,0.000369,0.000381,0.000201,0.003782,0.000542,0.000109,0.002568,0.045691,0.000140,0.000332,0.000700,0.000394,0.000153,0.000077,0.004837,0.000070,0.000053,0.001040,0.000478
-6h3K1hj0d4DRcZNUtHDuw,0.000447,0.050076,0.039643,0.001207,0.000276,0.001007,0.000561,0.000198,0.000451,0.000763,0.000184,0.000182,0.000459,0.003296,0.005615,0.001339,0.021692,0.000535,0.000293,0.000455,0.000455,0.000386,0.000791,0.000259,0.000811,0.081302,0.024390,0.000477,0.000151,0.000620,0.000305,0.007039,0.000729,0.001423,0.000456,0.000186,0.000066,0.000764,0.000146,0.074456,...,0.000188,0.000219,0.018085,0.000568,0.000355,0.043662,0.002309,0.071001,0.000354,0.000293,0.000812,0.001209,0.053601,0.000259,0.002993,0.000742,0.003843,0.007069,0.017197,0.000138,0.000221,0.002954,0.000515,0.000103,0.003216,0.000484,0.000670,0.027621,0.011289,0.000384,0.002316,0.002917,0.000074,0.000897,0.000926,0.000124,0.000099,0.000040,0.007819,0.001039
-8QlV3b_9H4BAh6LgMIr1g,0.000031,0.027002,0.043027,0.001043,0.000206,0.000046,0.000047,0.000084,0.000195,0.018048,0.000204,0.000020,0.000508,0.001096,0.000322,0.000293,0.005885,0.000107,0.000124,0.000168,0.000047,0.000032,0.002612,0.000023,0.000512,0.005953,0.010976,0.000163,0.000138,0.005799,0.000168,0.002407,0.000321,0.000604,0.000099,0.000040,0.000052,0.006688,0.000552,0.015662,...,0.000531,0.000498,0.000020,0.000136,0.000068,0.000618,0.031493,0.004186,0.000056,0.000203,0.000259,0.000063,0.011812,0.000172,0.000698,0.000111,0.00165

In [30]:
Q.iloc[0].sort_values(ascending=False).head(10)

matts        1.077482
breakfast    0.659543
wait         0.529304
food         0.428016
place        0.405557
eggs         0.362205
good         0.354656
bacon        0.312531
like         0.274301
big          0.265078
Name: -050d_XIor1NpCuWkbIVaQ, dtype: float64

Export Model

In [0]:
import pickle
output = open('yelp_recommendation_model_8.pkl', 'wb')
pickle.dump(P,output)
pickle.dump(Q,output)
pickle.dump(userid_vectorizer,output)
output.close()

Prediction for input text

In [0]:
words = "paneer butter masala"

In [69]:
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(text_process)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:7]

print("Text - ",words)
print("\n")
for i in topRecommendations.index:
    print(df_business[df_business['business_id']==i]['name'].iloc[0])
    print(df_business[df_business['business_id']==i]['categories'].iloc[0])
    print(str(df_business[df_business['business_id']==i]['stars'].iloc[0])+ ' '+str(df_business[df_business['business_id']==i]['review_count'].iloc[0]))
    print('')

Text -  paneer butter masala


Curry Corner
Pakistani, Halal, Restaurants, Indian
3.5 561

Chutney's Indian Cuisine
Indian, Buffets, Restaurants
4.0 337

Nandini Indian Cuisine
Restaurants, Buffets, Vegan, Indian
4.0 480

Woodlands Vegetarian South Indian Kitchen
Indian, Vegetarian, Pakistani, Restaurants
4.0 198

Marigold Maison- Paradise Valley
Indian, Restaurants
4.0 497

New India Gate
Restaurants, Buffets, Bars, Vegan, Halal, Nightlife, Beer Bar, Indian
4.5 474

India Oven
Restaurants, Buffets, Indian
4.5 709

